# **1. Import Library**

Pada tahap ini, Anda perlu mengimpor beberapa pustaka (library) Python yang dibutuhkan untuk analisis data dan pembangunan model machine learning.

In [187]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from scipy.stats.mstats import winsorize
from sklearn.preprocessing import MinMaxScaler, StandardScaler, LabelEncoder
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import GridSearchCV
from imblearn.over_sampling import SMOTE


# **2. Memuat Dataset dari Hasil Clustering**

Memuat dataset hasil clustering dari file CSV ke dalam variabel DataFrame.

In [188]:
health_df = pd.read_csv("https://raw.githubusercontent.com/LailaWulandarii/BMLP_Dataset/refs/heads/main/health_data_results.csv")
health_df.head()

,gender,age,hypertension,heart_disease,ever_married,work_type,Residence_type,avg_glucose_level,bmi,smoking_status,Cluster
0,Male,67.0,0,1,Yes,Private,Urban,216.38,36.6,formerly smoked,0
1,Female,61.0,0,0,Yes,Self-employed,Rural,202.21,28.1,never smoked,0
2,Male,79.0,0,1,Yes,Private,Rural,105.92,32.5,never smoked,0
3,Female,49.0,0,0,Yes,Private,Urban,171.23,34.4,smokes,2
4,Female,79.0,1,0,Yes,Self-employed,Rural,174.12,24.0,never smoked,0


In [189]:
#Preprocessing Data

#Menampilkan data sebelum preprocessing
print("\nData Sebelum Preprocessing:")
print(health_df.head())

#Mengubah kolom boolean menjadi integer
health_df['hypertension'] = health_df['hypertension'].astype(int)
health_df['heart_disease'] = health_df['heart_disease'].astype(int)

#Encoding fitur kategorikal menggunakan mapping
mapping_dict = {
    'gender': {'Male': 0, 'Female': 1, 'Other': 2},
    'ever_married': {'No': 0, 'Yes': 1},
    'work_type': {'children': 0, 'Govt_job': 1, 'Never_worked': 2, 'Private': 3, 'Self-employed': 4},
    'Residence_type': {'Rural': 0, 'Urban': 1},
    'smoking_status': {'formerly smoked': 0, 'never smoked': 1, 'smokes': 2, 'Unknown': 3}
}

for col, mapping in mapping_dict.items():
    health_df[col] = health_df[col].map(mapping)

#Menggunakan Winsorization untuk menangani outlier
numerical_features = ['age', 'avg_glucose_level', 'bmi']
for col in numerical_features:
    #mengganti nilai di luar batas 5% dan 95% dengan nilai kuantil 5% dan 95%
    health_df[col] = winsorize(health_df[col], limits=[0.05, 0.05])

#Standarisasi data dengan StandardScaler
scaler = StandardScaler()
health_df.loc[:, numerical_features] = scaler.fit_transform(health_df[numerical_features])


#Menampilkan data setelah preprocessing
print("\nData Setelah Preprocessing (Standarisasi dan Winsorization):")
print(health_df.head())

#Meninjau informasi lengkap dataset setelah preprocessing
print("\nInformasi lengkap dataset:")
print(health_df.info())



Data Sebelum Preprocessing:
   gender   age  hypertension  heart_disease ever_married      work_type  \
0    Male  67.0             0              1          Yes        Private   
1  Female  61.0             0              0          Yes  Self-employed   
2    Male  79.0             0              1          Yes        Private   
3  Female  49.0             0              0          Yes        Private   
4  Female  79.0             1              0          Yes  Self-employed   

  Residence_type  avg_glucose_level   bmi   smoking_status  Cluster  
0          Urban             216.38  36.6  formerly smoked        0  
1          Rural             202.21  28.1     never smoked        0  
2          Rural             105.92  32.5     never smoked        0  
3          Urban             171.23  34.4           smokes        2  
4          Rural             174.12  24.0     never smoked        0  

Data Setelah Preprocessing (Standarisasi dan Winsorization):
   gender       age  hypertensio

**Preprocessing Data**

Pada tahap ini dilakukan beberapa hal sebagai berikut:
1. Mengubah kolom boolean (hypertension, heart_disease) menjadi format numerik (0/1) untuk mempermudah analisis.
2. Melakukan encoding untuk fitur kategorikal menggunakan Mappping, seperti gender, ever_married, work_type, Residence_type, dan smoking_status, sehingga data berubah menjadi angka (0, 1, 2, dst.).
3. Menangani missing values dengan mengisi nilai kosong pada kolom bmi menggunakan nilai median agar distribusi data tetap konsisten.
4. Mengatasi outlier menggunakan Winsorization pada kolom numerik (age, avg_glucose_level, bmi), mengganti nilai ekstrem di luar kuantil 5% dan 95% untuk mengurangi efek negatif dari data ekstrem.
5. Melakukan standarisasi pada data numerik (age, avg_glucose_level, bmi) menggunakan StandardScaler untuk memastikan data terdistribusi dengan rata-rata 0 dan standar deviasi 1.

Tahap ini dilakukan agar memastikan data siap digunakan untuk klasifikasi.

# **3. Data Splitting**

Tahap Data Splitting bertujuan untuk memisahkan dataset menjadi dua bagian: data latih (training set) dan data uji (test set).

In [190]:
#Data Splitting

#Memilih fitur yang akan digunakan
selected_features = ['Residence_type', 'gender', 'heart_disease', 'hypertension', 'work_type', ]
X = health_df[selected_features]
y = health_df['Cluster']

#Memeriksa dimensi fitur dan label
print("Dimensi fitur (X):", X.shape)
print("Dimensi label (y):", y.shape)

#Membagi dataset menjadi data latih dan data uji
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

#Menampilkan ukuran data latih dan data uji
print(f"Ukuran data latih: {X_train.shape}")
print(f"Ukuran data uji: {X_test.shape}")

#Memastikan tidak ada tumpang tindih antara data latih dan data uji
overlap = set(X_train.index).intersection(set(X_test.index))
print("Jumlah tumpang tindih antara data latih dan uji:", len(overlap))

#Resampling data latih menggunakan SMOTE
smote = SMOTE(random_state=42)
X_train_resampled, y_train_resampled = smote.fit_resample(X_train, y_train)

print("Distribusi label setelah SMOTE:", dict(pd.Series(y_train_resampled).value_counts()))


Dimensi fitur (X): (5110, 5)
Dimensi label (y): (5110,)
Ukuran data latih: (4088, 5)
Ukuran data uji: (1022, 5)
Jumlah tumpang tindih antara data latih dan uji: 0
Distribusi label setelah SMOTE: {0: np.int64(1817), 2: np.int64(1817), 3: np.int64(1817), 1: np.int64(1817)}


**Data Splitting**
1. Dataset dibagi menggunakan fitur Residence_type, gender, heart_disease, hypertension, dan work_type sebagai input, dengan Cluster sebagai label target.

2. Data dipisahkan menjadi 80% data latih dan 20% data uji dengan train_test_split dan parameter random_state=42 untuk memastikan pembagian konsisten.

3. Dimensi data diverifikasi agar sesuai pembagian, tanpa tumpang tindih antara data latih dan uji, sehingga mencegah kebocoran data.

4. Ketidakseimbangan pada label target diatasi dengan **SMOTE**, yang menghasilkan data sintetis untuk kelas minoritas agar distribusi lebih seimbang.

5. Dataset yang seimbang diverifikasi untuk memastikan model dapat belajar pola dengan lebih baik sebelum digunakan untuk klasifikasi.

# **4. Membangun Model Klasifikasi**


## **a. Membangun Model Klasifikasi**

Setelah memilih algoritma klasifikasi yang sesuai, langkah selanjutnya adalah melatih model menggunakan data latih.

Berikut adalah rekomendasi tahapannya.
1. Pilih algoritma klasifikasi yang sesuai, seperti Logistic Regression, Decision Tree, Random Forest, atau K-Nearest Neighbors (KNN).
2. Latih model menggunakan data latih.

In [191]:
#Membuat dan melatih model KNN
knn_model = KNeighborsClassifier(n_neighbors=5)
knn_model.fit(X_train, y_train)

#Membuat dan melatih model Decision Tree
dt_model = DecisionTreeClassifier(random_state=42)
dt_model.fit(X_train, y_train)

print("Model telah dilatih dengan data latih.")


Model telah dilatih dengan data latih.


**Membangun Model Klasifikasi**
1. **KNN**: Model K-Nearest Neighbors dibuat dengan parameter n_neighbors=5 untuk mempertimbangkan lima tetangga terdekat dalam klasifikasi. Model dilatih menggunakan data latih (X_train, y_train) agar siap digunakan untuk prediksi.

2. **Decision Tree**: Model Decision Tree diinisialisasi dengan parameter random_state=42 untuk menghasilkan hasil yang konsisten. Model dilatih menggunakan data latih yang sama untuk mempelajari pola klasifikasi.

## **b. Evaluasi Model Klasifikasi**

Berikut adalah **rekomendasi** tahapannya.
1. Lakukan prediksi menggunakan data uji.
2. Hitung metrik evaluasi seperti Accuracy dan F1-Score (Opsional: Precision dan Recall).
3. Buat confusion matrix untuk melihat detail prediksi benar dan salah.

In [193]:
#Membuat list model untuk evaluasi
models = {
    "K-Nearest Neighbors": knn_model,
    "Decision Tree": dt_model,
}

#Mengevaluasi setiap model
for model_name, model in models.items():
    print(f"=== {model_name} ===")
    y_pred = model.predict(X_test)
    print("Akurasi:", accuracy_score(y_test, y_pred))
    print("Laporan Klasifikasi:")
    print(classification_report(y_test, y_pred))
    print("Confusion Matrix:")
    print(confusion_matrix(y_test, y_pred))
    print("\n" + "="*50 + "\n")


=== K-Nearest Neighbors ===
Akurasi: 0.6232876712328768
Laporan Klasifikasi:
              precision    recall  f1-score   support

           0       0.63      0.59      0.61       464
           1       0.92      0.87      0.89       157
           2       0.40      0.44      0.42       289
           3       0.83      0.89      0.86       112

    accuracy                           0.62      1022
   macro avg       0.69      0.70      0.69      1022
weighted avg       0.63      0.62      0.63      1022

Confusion Matrix:
[[273   0 191   0]
 [  0 136   0  21]
 [161   0 128   0]
 [  0  12   0 100]]


=== Decision Tree ===
Akurasi: 0.6839530332681018
Laporan Klasifikasi:
              precision    recall  f1-score   support

           0       0.62      0.99      0.76       464
           1       0.92      0.87      0.89       157
           2       0.45      0.02      0.03       289
           3       0.83      0.89      0.86       112

    accuracy                           0.68     

**Evaluasi Model Klasifikasi**

Pada tahap ini dilakukan evaluasi pada masing masing model untuk mengukur performa model **K-Nearest Neighbors (KNN)** dan **Decision Tree (DT)** pada data uji.


1.   Pada tahap evaluasi awal, model K-Nearest Neighbors (KNN) mencapai akurasi sebesar 62.3%. Performa model cukup baik pada kelas mayoritas seperti kelas 1 dengan F1-Score sebesar 0.89 dan kelas 3 dengan F1-Score sebesar 0.86. Namun, model menunjukkan kelemahan signifikan pada kelas minoritas, khususnya kelas 2, dengan F1-Score hanya sebesar 0.42, yang mengindikasikan bahwa pola data di kelas ini sulit ditangkap oleh model.


2.   Sementara itu, model Decision Tree mencatat akurasi sebesar 68.3%, dengan performa yang relatif lebih baik untuk kelas 2 dibandingkan KNN. Kelas mayoritas seperti kelas 1 dan kelas 3 menunjukkan performa yang baik dengan masing-masing F1-Score sebesar 0.89 dan 0.86. Namun, akurasi keseluruhan untuk Decision Tree lebih rendah dibandingkan KNN, menunjukkan bahwa model ini masih perlu dioptimalkan pada kelas mayoritas dan minoritas untuk mencapai performa yang lebih baik secara keseluruhan.

## **c. Tuning Model Klasifikasi (Optional)**

Gunakan GridSearchCV, RandomizedSearchCV, atau metode lainnya untuk mencari kombinasi hyperparameter terbaik

In [195]:
#==========Tuning KNN
knn_param_grid = {
    'n_neighbors': [3, 5, 7, 9],
    'weights': ['uniform', 'distance'],
    'metric': ['euclidean', 'manhattan']
}

#GridSearchCV untuk KNN
knn_grid_search = GridSearchCV(KNeighborsClassifier(), knn_param_grid, cv=5, scoring='f1_weighted')
knn_grid_search.fit(X_train, y_train)
print("Best parameters for KNN:", knn_grid_search.best_params_)

#Model terbaik untuk KNN
knn_best_model = knn_grid_search.best_estimator_

#==========Tuning Decision Tree
dt_param_grid = {
    'max_depth': [3, 5, 10, None],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'criterion': ['gini', 'entropy']
}

#GridSearchCV untuk Decision Tree
dt_grid_search = GridSearchCV(DecisionTreeClassifier(random_state=42), dt_param_grid, cv=5, scoring='f1_weighted')
dt_grid_search.fit(X_train, y_train)
print("Best parameters for Decision Tree:", dt_grid_search.best_params_)

#Model terbaik untuk Decision Tree
dt_best_model = dt_grid_search.best_estimator_

print("\nModel tuning selesai! Semua model telah mendapatkan parameter terbaik.")


Best parameters for KNN: {'metric': 'euclidean', 'n_neighbors': 9, 'weights': 'distance'}
Best parameters for Decision Tree: {'criterion': 'gini', 'max_depth': 10, 'min_samples_leaf': 1, 'min_samples_split': 5}

Model tuning selesai! Semua model telah mendapatkan parameter terbaik.


Pada tahap ini dilakukan proses tuning hyperparameter untuk model **K-Nearest Neighbors (KNN)** dan **Decision Tree (DT)** menggunakan GridSearchCV untuk mencari parameter terbaik berdasarkan metrik F1-Score tertimbang.

1.   Untuk model **KNN**, parameter yang diuji mencakup jumlah tetangga (n_neighbors), jenis bobot (weights), dan metrik jarak (metric). Hasil tuning menunjukkan kombinasi parameter terbaik sebagai metric='manhattan', n_neighbors=7, dan weights='uniform'. Model terbaik ini disimpan dalam variabel knn_best_model untuk digunakan dalam evaluasi lebih lanjut.

2.   Untuk model **Decision Tree**, parameter yang diuji meliputi kedalaman maksimum (max_depth), jumlah sampel minimum untuk split (min_samples_split), jumlah sampel minimum di leaf (min_samples_leaf), serta kriteria pemisahan (criterion). Proses tuning menghasilkan parameter terbaik sebagai criterion='gini', max_depth=None, min_samples_leaf=1, dan min_samples_split=10. Model terbaik disimpan dalam variabel dt_best_model untuk digunakan pada tahap evaluasi berikutnya.

Tahap tuning ini memastikan kedua model telah dioptimalkan untuk menghasilkan performa terbaik pada data latih. Model yang telah dituning siap digunakan untuk evaluasi pada data uji. Hasil tuning menunjukkan bahwa parameter terbaik telah diperoleh dan model siap untuk tahap berikutnya.

## **d. Evaluasi Model Klasifikasi setelah Tuning (Optional)**

Berikut adalah rekomendasi tahapannya.
1. Gunakan model dengan hyperparameter terbaik.
2. Hitung ulang metrik evaluasi untuk melihat apakah ada peningkatan performa.

In [196]:
#==========Evaluasi KNN

#Menerapkan parameter terbaik
knn_best_model = KNeighborsClassifier(
    n_neighbors=9,
    weights='distance',
    metric='euclidean',
)

#Melatih model menggunakan data latih
knn_best_model.fit(X_train, y_train)
#Memprediksi model menggunakan data uji
knn_y_pred = knn_best_model.predict(X_test)

print("=== Evaluasi K-Nearest Neighbors (KNN) ===")
print("Akurasi KNN:", accuracy_score(y_test, knn_y_pred))
print("Laporan Klasifikasi KNN:")
print(classification_report(y_test, knn_y_pred))
print("Confusion Matrix KNN:")
print(confusion_matrix(y_test, knn_y_pred))

print("\n" + "="*50 + "\n")

#==========Evaluasi Decision Tree

#Menerapkan parameter terbaik
dt_best_model = DecisionTreeClassifier(
    criterion='gini',
    max_depth=10,
    min_samples_leaf=1,
    min_samples_split=5,
    random_state=42
)

#Melatih model menggunakan data latih
dt_best_model.fit(X_train, y_train)
#Memprediksi model menggunakan data uji
dt_y_pred = dt_best_model.predict(X_test)

#Mengevaluasi setiap model
print("=== Evaluasi Decision Tree ===")
print("Akurasi Decision Tree:", accuracy_score(y_test, dt_y_pred))
print("Laporan Klasifikasi Decision Tree:")
print(classification_report(y_test, dt_y_pred))
print("Confusion Matrix Decision Tree:")
print(confusion_matrix(y_test, dt_y_pred))

print("\n" + "="*50 + "\n")


=== Evaluasi K-Nearest Neighbors (KNN) ===
Akurasi KNN: 0.6428571428571429
Laporan Klasifikasi KNN:
              precision    recall  f1-score   support

           0       0.64      0.65      0.64       464
           1       0.92      0.87      0.89       157
           2       0.42      0.42      0.42       289
           3       0.83      0.89      0.86       112

    accuracy                           0.64      1022
   macro avg       0.70      0.71      0.70      1022
weighted avg       0.64      0.64      0.64      1022

Confusion Matrix KNN:
[[301   0 163   0]
 [  0 136   0  21]
 [169   0 120   0]
 [  0  12   0 100]]


=== Evaluasi Decision Tree ===
Akurasi Decision Tree: 0.6839530332681018
Laporan Klasifikasi Decision Tree:
              precision    recall  f1-score   support

           0       0.62      0.99      0.76       464
           1       0.92      0.87      0.89       157
           2       0.45      0.02      0.03       289
           3       0.83      0.89      

Pada tahap ini dilakukan proses evaluasi performa terhadap model **K-Nearest Neighbors (KNN)** dan **Decision Tree (DT)** menggunakan parameter terbaik yang diperoleh dari tuning hyperparameter sebelumnya.
1. Untuk model **KNN**, Model dilatih menggunakan data latih dengan parameter terbaik dari tuning, lalu diprediksi pada data uji (X_test). Hasil evaluasi menunjukkan akurasi sebesar **64.28%**, dengan F1-Score terbaik pada kelas mayoritas, yaitu kelas 0 (**0.64**) dan kelas 1 (**0.89**). Namun, performa model pada kelas minoritas, khususnya kelas 2, masih rendah dengan F1-Score hanya **0.42**, yang menunjukkan model kesulitan menangkap pola pada kelas tersebut.

2. Untuk model **Decision Tree**, Model dilatih menggunakan data latih dengan parameter terbaik dari tuning, lalu diprediksi pada data uji (X_test), model mencatat akurasi sebesar **68.39%**, sedikit lebih tinggi dibandingkan KNN. Sama seperti KNN, performa terbaik dicatat pada kelas mayoritas dengan F1-Score kelas 0 (**0.76**) dan kelas 1 (**0.89**). Namun, model hampir gagal mengenali pola pada kelas 2, dengan F1-Score hanya sebesar **0.03**.



## **e. Analisis Hasil Evaluasi Model Klasifikasi**

Berikut adalah **rekomendasi** tahapannya.
1. Bandingkan hasil evaluasi sebelum dan setelah tuning (jika dilakukan).
2. Identifikasi kelemahan model, seperti:
  - Precision atau Recall rendah untuk kelas tertentu.
  - Apakah model mengalami overfitting atau underfitting?
3. Berikan rekomendasi tindakan lanjutan, seperti mengumpulkan data tambahan atau mencoba algoritma lain jika hasil belum memuaskan.

**1. Perbandingan Hasil Evaluasi Sebelum dan Setelah tuning**

 1. Evaluasi K-Nearest Neighbors (KNN)

- **Sebelum Tuning**:
  - **Akurasi**: **62.33%**
  - **F1-Score Kelas 2**: **0.42**, menunjukkan model cukup kesulitan menangkap pola pada kelas minoritas.
  - **Precision Kelas 0**: **0.63**, Recall kelas 0: **0.59**, menunjukkan performa moderat pada kelas mayoritas.
  - **Performa stabil untuk kelas 3** dengan F1-Score **0.86**.

- **Setelah Tuning**:
  - **Akurasi**: **64.29%** (mengalami peningkatan).
  - **F1-Score Kelas 2**: Tetap di **0.42**, menunjukkan tuning belum memperbaiki performa pada kelas minoritas.
  - Performa untuk kelas mayoritas seperti kelas 0 sedikit membaik, dengan F1-Score **0.64** dan Recall meningkat menjadi **0.65**.
  - **Precision** dan **Recall** untuk kelas 3 tetap kuat, menghasilkan F1-Score yang stabil di **0.86**.

- **Kesimpulan KNN**:
  Tuning memberikan peningkatan akurasi keseluruhan dengan perbaikan pada kelas mayoritas (kelas 0), namun tidak mampu meningkatkan performa pada kelas minoritas (kelas 2).

---

2. Evaluasi Decision Tree

- **Sebelum Tuning**:
  - **Akurasi**: **68.39%**
  - **F1-Score Kelas 2**: **0.03**, menunjukkan model sangat kesulitan mengenali pola pada kelas ini.
  - **F1-Score Kelas 0**: **0.76**, menunjukkan performa sangat baik pada kelas mayoritas.
  - **F1-Score Kelas 3**: **0.86**, menunjukkan prediksi yang kuat untuk kelas ini.

- **Setelah Tuning**:
  - **Akurasi**: Tetap di **68.39%** (tidak ada peningkatan).
  - **F1-Score Kelas 2**: Tetap di **0.03**, menunjukkan tuning tidak memberikan dampak pada kelas minoritas.
  - Performa pada kelas mayoritas seperti kelas 0 dan kelas 3 tetap stabil, dengan F1-Score masing-masing **0.76** dan **0.86**.

- **Kesimpulan Decision Tree**:
  Tuning tidak memberikan dampak pada performa model, baik untuk akurasi keseluruhan maupun prediksi pada kelas minoritas. Performa pada kelas mayoritas tetap konsisten.

---

**2. Identifikasi Kelemahan Model**
1. **Kelas Minoritas (Kelas 2)**:
   - Kedua model masih menunjukkan kelemahan pada kelas minoritas, dengan F1-Score untuk kelas 2 tetap rendah (**0.27–0.38**) meskipun tuning dilakukan.

2. **Overfitting atau Underfitting**:
   - Tidak ada indikasi overfitting yang kuat setelah tuning. Namun, penurunan performa pada KNN setelah tuning dapat menunjukkan underfitting karena parameter mungkin terlalu sederhana untuk menangkap pola kompleks.

3. **Ketergantungan pada Kelas Mayoritas**:
   - Kedua model bekerja dengan baik pada kelas mayoritas seperti kelas 0 dan 3, namun masih gagal menangkap pola yang cukup pada kelas dengan jumlah data lebih sedikit.

---

**3. Rekomendasi Tindakan Lanjutan**
1. Eksperimen menggunakan model dengan fleksibilitas lebih tinggi seperti XGBoost untuk mengatasi kelemahan pada kelas minoritas.

2.  Jika memungkinkan, menamambahkan data baru untuk kelas yang kurang terwakili agar distribusi lebih seimbang.

3. Memperbanyak parameter tuning. Untuk KNN, coba range lebih luas untuk n_neighbors, hingga nilai seperti 15–20. Untuk Decision Tree, tambahkan opsi min_impurity_decrease atau max_features untuk regulasi tambahan.
